Dieses Notebook enthält die Daten aus detected_syringes und dected_syringes with bubbles

In [6]:
import os
import glob
import cv2 as cv
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import joblib

# --- Paths ---
ok_folder = "detected_syringes"  # OK-class ROIs
defect_folder = "detected_syringes_with_bubbles"  # Defect-class ROIs

# --- Feature extraction function ---
def extract_features(image_path):
    img = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    if img is None:
        return None
    img = cv.resize(img, (64, 64))
    blur = cv.GaussianBlur(img, (5, 5), 0)
    edges = cv.Canny(blur, 30, 100)
    hist = cv.calcHist([img], [0], None, [16], [0, 256]).flatten()
    return np.concatenate([edges.flatten(), hist])

# --- Load data ---
features = []
labels = []

for filepath in glob.glob(os.path.join(ok_folder, "*.jpg")):
    f = extract_features(filepath)
    if f is not None:
        features.append(f)
        labels.append("ok")

for filepath in glob.glob(os.path.join(defect_folder, "*.jpg")):
    f = extract_features(filepath)
    if f is not None:
        features.append(f)
        labels.append("defect")

# --- Prepare data ---
X = np.array(features)
y = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Train unweighted model ---
clf_unweighted = RandomForestClassifier(n_estimators=100, random_state=42)
clf_unweighted.fit(X_train, y_train)
y_pred_unweighted = clf_unweighted.predict(X_test)

print("\n--- Classification Report (Unweighted) ---")
print(classification_report(y_test, y_pred_unweighted, target_names=["defect", "ok"]))

ConfusionMatrixDisplay.from_predictions(y_test, y_pred_unweighted, display_labels=["defect", "ok"], cmap="Blues")
plt.title("Confusion Matrix – Unweighted Model")
plt.grid(False)
plt.show()

# --- Train weighted model ---
clf_weighted = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=42)
clf_weighted.fit(X_train, y_train)
y_pred_weighted = clf_weighted.predict(X_test)

print("\n--- Classification Report (Weighted) ---")
print(classification_report(y_test, y_pred_weighted, target_names=["defect", "ok"]))

ConfusionMatrixDisplay.from_predictions(y_test, y_pred_weighted, display_labels=["defect", "ok"], cmap="Greens")
plt.title("Confusion Matrix – Weighted Model")
plt.grid(False)
plt.show()

# --- Export weighted model ---
joblib.dump(clf_weighted, "syringe_model_roi_weighted.pkl")
print("\nWeighted model saved as syringe_model_roi_weighted.pkl")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\haede\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\haede\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\haede\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: initialization failed